In [1]:
# Imports.
import re
import requests
import pandas as pd

In [38]:
df_intelligence = pd.read_csv('dog_intelligence.csv')
df_ranks = pd.read_csv('breed_rank.csv')
df_traits = pd.read_csv('breed_traits.csv')
df_sizes = pd.read_csv('akc_breed_info.csv')

## Clean Intelligence Data

1. Remove NaN.
2. Rename columns to more sensible lowercase names.
3. Make breed names lowercase.
4. Simplify intelligence classification from strings to numbers [1, 5].
5. Make obey percent column numeric.
6. Combine lower and upper reps column into 1 reps_avg column.
7. Replace breed name 'Petit Basset Griffon VendÌeen' with 'Petit Basset Griffon Vendeen'.
8. Drop row with breeds 'cocker spaniel' and 'king charles spaniel' as these are redundant.

In [36]:
# 1. Remove NaN.
df_intelligence = df_intelligence.dropna().reset_index(drop=True)

In [33]:
# 2. Rename columns to more sensible lowercase names.
df_intelligence.columns = [
    'breed', 'working_intelligence_level', 
    'obedience_pc', 'reps_lower', 'reps_upper'
]

In [34]:
# 3. Make breed names lowercase.
for i in range(len(df_intelligence)):
    df_intelligence.loc[i, 'breed'] = df_intelligence.loc[i, 'breed'].lower()

In [22]:
# 4. Simplify intelligence classification from strings to numbers [1, 5].
working_intelligence_levels = {
    'Fair Working/Obedience Intelligence': 1,
    'Average Working/Obedience Intelligence': 2,
    'Above Average Working Dogs': 3, 
    'Excellent Working Dogs': 4, 
    'Brightest Dogs': 5
}
for i in range(len(df_intelligence)):
    v = df_intelligence.loc[i, 'working_intelligence_level']
    df_intelligence.loc[i, 'working_intelligence_level'] = working_intelligence_levels[v]

In [23]:
# 5. Make obey percent column numeric.
for i in range(len(df_intelligence)):
    v = df_intelligence.loc[i, 'obedience_pc']
    df_intelligence.loc[i, 'obedience_pc'] = int(v.replace('%', ''))

In [24]:
# 6. Combine lower and upper reps column into 1 reps_avg column.
df_intelligence = pd.concat([
    df_intelligence.drop(['reps_lower', 'reps_upper'], axis=1), 
    pd.DataFrame({'reps_avg': df_intelligence[['reps_lower', 'reps_upper']].mean(axis=1)})
], axis=1)

In [25]:
# 7. Replace breed name 'Petit Basset Griffon VendÌ©en' with 'Petit Basset Griffon Vendeen'.
for i in range(len(df_intelligence['breed'])):
    b = df_intelligence['breed'][i]
    if 'Petit Basset Griffon' in b:
        df_intelligence.loc[i, 'breed'] = 'petit basset griffon vendeen'

In [26]:
# 8. Drop row with breeds 'cocker spaniel' and 'king charles spaniel' as these are redundant.
df_intelligence.drop([
    df_intelligence[df_intelligence['breed'] == 'cocker spaniel'].index[0],
    df_intelligence[df_intelligence['breed'] == 'king charles spaniel'].index[0],
], axis=0, inplace=True)

In [27]:
df_intelligence

,breed,working_intelligence_level,obedience_pc,reps_avg
0,border collie,5,95,2.5
1,poodle,5,95,2.5
2,german shepherd,5,95,2.5
3,golden retriever,5,95,2.5
4,doberman pinscher,5,95,2.5
...,...,...,...,...
120,saint bernard,1,30,60.5
121,bull terrier,1,30,60.5
122,chihuahua,1,30,60.5
123,lhasa apso,1,30,60.5


In [29]:
for breed in df_intelligence['breed']:
    if 'hih' in breed: print(breed)

chihuahua


## Clean Ranks Data

1. Remove NaN values.
2. Drop links column.
3. Rename columns to more sensible lowercase names.
4. Make breed names lowercase.

In [16]:
df_ranks

,Breed,2013 Rank,2014 Rank,2015 Rank,2016 Rank,2017 Rank,2018 Rank,2019 Rank,2020 Rank,links,Image
0,Retrievers (Labrador),1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,https://www.akc.org/dog-breeds/labrador-retrie...,https://www.akc.org/wp-content/uploads/2017/11...
1,French Bulldogs,11.0,9.0,6.0,6.0,4.0,4.0,4.0,2,https://www.akc.org/dog-breeds/french-bulldog/,https://www.akc.org/wp-content/uploads/2017/11...
2,German Shepherd Dogs,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3,https://www.akc.org/dog-breeds/german-shepherd...,https://www.akc.org/wp-content/uploads/2017/11...
3,Retrievers (Golden),3.0,3.0,3.0,3.0,3.0,3.0,3.0,4,https://www.akc.org/dog-breeds/golden-retriever/,https://www.akc.org/wp-content/uploads/2017/11...
4,Bulldogs,5.0,4.0,4.0,4.0,5.0,5.0,5.0,5,https://www.akc.org/dog-breeds/bulldog/,https://www.akc.org/wp-content/uploads/2017/11...
...,...,...,...,...,...,...,...,...,...,...,...
190,Cesky Terriers,174.0,182.0,179.0,182.0,185.0,185.0,190.0,191,https://www.akc.org/dog-breeds/cesky-terrier/,https://www.akc.org/wp-content/uploads/2017/11...
191,American Foxhounds,176.0,180.0,181.0,189.0,187.0,186.0,191.0,192,https://www.akc.org/dog-breeds/american-foxhound/,https://www.akc.org/wp-content/uploads/2017/11...
192,Azawakhs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,193,https://www.akc.org/dog-breeds/azawakh/,https://www.akc.org/wp-content/uploads/2017/11...
193,English Foxhounds,177.0,183.0,184.0,187.0,189.0,188.0,195.0,194,https://www.akc.org/dog-breeds/english-foxhound/,https://www.akc.org/wp-content/uploads/2017/11...


In [17]:
# 1. Remove NaN values.
df_ranks = df_ranks.dropna().reset_index(drop=True)

In [18]:
# 2. Drop links column.
df_ranks.drop(['links'], axis=1, inplace=True)

In [19]:
# 3. Rename columns to more sensible lowercase names.
df_ranks.columns = ['breed'] + [f'rank_{year}' for year in range(2013, 2021)] + ['image']

In [20]:
# 4. Make breed names lowercase.
for i in range(len(df_ranks)):
    df_ranks.loc[i, 'breed'] = df_ranks.loc[i, 'breed'].lower()

In [21]:
df_ranks

,breed,rank_2013,rank_2014,rank_2015,rank_2016,rank_2017,rank_2018,rank_2019,rank_2020,image
0,retrievers (labrador),1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,https://www.akc.org/wp-content/uploads/2017/11...
1,french bulldogs,11.0,9.0,6.0,6.0,4.0,4.0,4.0,2,https://www.akc.org/wp-content/uploads/2017/11...
2,german shepherd dogs,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3,https://www.akc.org/wp-content/uploads/2017/11...
3,retrievers (golden),3.0,3.0,3.0,3.0,3.0,3.0,3.0,4,https://www.akc.org/wp-content/uploads/2017/11...
4,bulldogs,5.0,4.0,4.0,4.0,5.0,5.0,5.0,5,https://www.akc.org/wp-content/uploads/2017/11...
...,...,...,...,...,...,...,...,...,...,...
170,harriers,173.0,181.0,183.0,186.0,183.0,189.0,188.0,190,https://www.akc.org/wp-content/uploads/2017/11...
171,cesky terriers,174.0,182.0,179.0,182.0,185.0,185.0,190.0,191,https://www.akc.org/wp-content/uploads/2017/11...
172,american foxhounds,176.0,180.0,181.0,189.0,187.0,186.0,191.0,192,https://www.akc.org/wp-content/uploads/2017/11...
173,english foxhounds,177.0,183.0,184.0,187.0,189.0,188.0,195.0,194,https://www.akc.org/wp-content/uploads/2017/11...


## Clean Traits Data

1. Rename columns to more sensible lowercase names.
2. Make breed names, coat type and coat length values lowercase.

In [22]:
df_traits

,Breed,Affectionate With Family,Good With Young Children,Good With Other Dogs,Shedding Level,Coat Grooming Frequency,Drooling Level,Coat Type,Coat Length,Openness To Strangers,Playfulness Level,Watchdog/Protective Nature,Adaptability Level,Trainability Level,Energy Level,Barking Level,Mental Stimulation Needs
0,Retrievers (Labrador),5,5,5,4,2,2,Double,Short,5,5,3,5,5,5,3,4
1,French Bulldogs,5,5,4,3,1,3,Smooth,Short,5,5,3,5,4,3,1,3
2,German Shepherd Dogs,5,5,3,4,2,2,Double,Medium,3,4,5,5,5,5,3,5
3,Retrievers (Golden),5,5,5,4,2,2,Double,Medium,5,4,3,5,5,3,1,4
4,Bulldogs,4,3,3,3,3,3,Smooth,Short,4,4,3,3,4,3,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,Cesky Terriers,4,5,3,2,2,1,Wavy,Medium,4,3,3,4,3,3,3,3
191,American Foxhounds,3,5,5,3,1,1,Smooth,Short,3,3,3,3,3,4,5,3
192,Azawakhs,3,3,3,2,2,1,Smooth,Short,1,3,3,3,2,3,1,3
193,English Foxhounds,5,5,5,3,1,2,Double,Short,4,4,3,4,4,4,5,4


In [23]:
# 1. Rename columns to more sensible lowercase names.
df_traits.columns = [
    'breed', 'affection_level', 'child_friendliness', 'dog_friendliness', 'shedding_level',
    'grooming_frequency', 'drooling_level', 'coat_type', 'coat_length', 
    'openness_to_strangers', 'playfulness', 'protectiveness', 'adaptability', 
    'trainability', 'energy_level', 'barking_level', 'mental_stimulation_needs'
]

In [24]:
# 2. Make breed names, coat type and coat length values lowercase.
for i in range(len(df_traits)):
    df_traits.loc[i, 'breed'] = df_traits.loc[i, 'breed'].lower()
    df_traits.loc[i, 'coat_type'] = df_traits.loc[i, 'coat_type'].lower()
    df_traits.loc[i, 'coat_length'] = df_traits.loc[i, 'coat_length'].lower()

In [25]:
df_traits

,breed,affection_level,child_friendliness,dog_friendliness,shedding_level,grooming_frequency,drooling_level,coat_type,coat_length,openness_to_strangers,playfulness,protectiveness,adaptability,trainability,energy_level,barking_level,mental_stimulation_needs
0,retrievers (labrador),5,5,5,4,2,2,double,short,5,5,3,5,5,5,3,4
1,french bulldogs,5,5,4,3,1,3,smooth,short,5,5,3,5,4,3,1,3
2,german shepherd dogs,5,5,3,4,2,2,double,medium,3,4,5,5,5,5,3,5
3,retrievers (golden),5,5,5,4,2,2,double,medium,5,4,3,5,5,3,1,4
4,bulldogs,4,3,3,3,3,3,smooth,short,4,4,3,3,4,3,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,cesky terriers,4,5,3,2,2,1,wavy,medium,4,3,3,4,3,3,3,3
191,american foxhounds,3,5,5,3,1,1,smooth,short,3,3,3,3,3,4,5,3
192,azawakhs,3,3,3,2,2,1,smooth,short,1,3,3,3,2,3,1,3
193,english foxhounds,5,5,5,3,1,2,double,short,4,4,3,4,4,4,5,4


## Clean Sizes Data

1. Compute average height and weight instead of upper and lower bounds.
2. Make breed names, coat type and coat length values lowercase.
3. Add size categories by weight as follows (https://vetcarenews.com/small-medium-large-dog-size-by-weight-guide/).
    * Teacup =< 4 lbs.
    * Toy = [5, 11] lbs.
    * Small = [12, 23] lbs.
    * Medium = [24, 58] lbs.
    * Large = [59, 99] lbs.
    * Giant >= 100 lbs

In [26]:
df_sizes

,Breed,height_low_inches,height_high_inches,weight_low_lbs,weight_high_lbs
0,Akita,26.0,28.0,80,120
1,Anatolian Sheepdog,27.0,29.0,100,150
2,Bernese Mountain Dog,23.0,27.0,85,110
3,Bloodhound,24.0,26.0,80,120
4,Borzoi,26.0,28.0,70,100
...,...,...,...,...,...
143,Papillon,8.0,11.0,5,10
144,Pomeranian,12.0,12.0,3,7
145,Poodle Toy,10.0,10.0,10,10
146,Toy Fox Terrier,10.0,10.0,4,7


In [27]:
# 1. Compute average height and weight instead of upper and lower bounds.
df_sizes = pd.DataFrame({
    'breed': df_sizes['Breed'],
    'height_avg_in': df_sizes[['height_low_inches', 'height_high_inches']].mean(axis=1),
    'weight_avg_lbs': df_sizes[['weight_low_lbs', 'weight_high_lbs']].mean(axis=1)
})

In [28]:
# 2. Make breed names, coat type and coat length values lowercase.
for i in range(len(df_sizes)):
    df_sizes.loc[i, 'breed'] = df_sizes.loc[i, 'breed'].lower()

In [29]:
# 3. Add size categories by weight.
weights = df_sizes['weight_avg_lbs']
sizes = []
for lbs in weights:
    if lbs < 5: sizes.append('teacup')
    elif lbs < 12: sizes.append('toy')
    elif lbs < 24: sizes.append('small')
    elif lbs < 59: sizes.append('medium')
    elif lbs < 100: sizes.append('large')
    else: sizes.append('giant')
df_sizes['size'] = sizes

In [30]:
df_sizes

,breed,height_avg_in,weight_avg_lbs,size
0,akita,27.0,100.0,giant
1,anatolian sheepdog,28.0,125.0,giant
2,bernese mountain dog,25.0,97.5,large
3,bloodhound,25.0,100.0,giant
4,borzoi,27.0,85.0,large
...,...,...,...,...
143,papillon,9.5,7.5,toy
144,pomeranian,12.0,5.0,toy
145,poodle toy,10.0,10.0,toy
146,toy fox terrier,10.0,5.5,toy


All 4 datasets have information corresponding to similar breeds. But the breed name is written differently among the 4 datasets. Also, intelligence data is available for fewer breeds compare to other attributes.

Only the breeds in the intelligence datasets shall be extracted so that all data is available for visualization.

This section shall also replace breed names in all 4 datasets so that the each breed is known by the same name across all 4 datasets.

## Resolving Breed Name Differences

In [31]:
def keep_alphanum_only(s):
    ''' Removes any character that is not an alphabet or number. '''
    s = ' '.join([w for w in re.sub('[^0-9a-zA-Z]+', ' ', s).split(' ') if len(w)>0])
    return s

In [32]:
def search_breed(breed, breed_list):
    ''' Searches for possibly alternate name for the same given breed 
        in given breed list and returns this name if found or '' otherwise.
    '''
    breed_processed = keep_alphanum_only(breed.lower())

    matches = []
    matches_processed = []
    for b in breed_list:
        # Check if b has all the words that given breed has.
        b_processed = keep_alphanum_only(b.lower())
        has_all_words = True
        for w in breed_processed.split(): 
            if not w in b_processed: 
                has_all_words = False
                break
        if has_all_words: 
            matches.append(b)
            matches_processed.append(b_processed)

    min_len_diff = float('inf')
    best_match = ''
    for i in range(len(matches_processed)):
        m_processed = matches_processed[i]
        len_diff = abs(len(breed_processed) - len(m_processed))
        if len_diff < min_len_diff:
            min_len_diff = len_diff
            best_match = matches[i]
    
    return best_match

In [33]:
breeds = {'intelligence':[], 'ranks':[], 'sizes':[], 'traits':[]}
# Iterate over intelligence dataset because it is the smallest and hence it is 
# likely that all the breeds that it contains is also present in all the other datasets.
for bi in df_intelligence['breed']:
    # Find breed name corresponding to this breed in the ranks dataset.
    best_match_ranks = search_breed(bi, df_ranks['breed'])
    # Find breed name corresponding to this breed in the traits dataset.
    best_match_traits = search_breed(bi, df_traits['breed'])
    # Find breed name corresponding to this breed in the sizes dataset.
    best_match_sizes = search_breed(bi, df_sizes['breed'])
    # Consider breed if there was a match among all 4 datasets.
    if best_match_ranks != '' and best_match_traits != '' and best_match_sizes != '':
        breeds['intelligence'].append(bi)
        breeds['ranks'].append(best_match_ranks)
        breeds['traits'].append(best_match_traits)
        breeds['sizes'].append(best_match_sizes)
df_breed_names = pd.DataFrame(breeds)

In [34]:
# Make breed names consistent across datasets.
for index, row in df_breed_names.iterrows():
    df_ranks = df_ranks.replace(to_replace=row['ranks'], value=row['intelligence'])
    df_traits = df_traits.replace(to_replace=row['traits'], value=row['intelligence'])
    df_sizes = df_sizes.replace(to_replace=row['sizes'], value=row['intelligence'])

In [35]:
# Join datasets.
df_dogs = df_intelligence.merge(df_ranks, on='breed')
df_dogs = df_dogs.merge(df_traits, on='breed')
df_dogs = df_dogs.merge(df_sizes, on='breed')

In [36]:
df_dogs

,breed,working_intelligence_level,obedience_pc,reps_avg,rank_2013,rank_2014,rank_2015,rank_2016,rank_2017,rank_2018,...,playfulness,protectiveness,adaptability,trainability,energy_level,barking_level,mental_stimulation_needs,height_avg_in,weight_avg_lbs,size
0,poodle,5,95,2.5,8.0,7.0,8.0,7.0,7.0,7.0,...,5,5,4,5,4,4,5,10.0,10.0,toy
1,german shepherd,5,95,2.5,2.0,2.0,2.0,2.0,2.0,2.0,...,4,5,5,5,5,3,5,24.0,82.5,large
2,golden retriever,5,95,2.5,3.0,3.0,3.0,3.0,3.0,3.0,...,4,3,5,5,3,1,4,22.5,65.0,large
3,doberman pinscher,5,95,2.5,12.0,14.0,14.0,15.0,16.0,17.0,...,4,5,4,5,5,3,4,27.0,80.0,large
4,shetland sheepdog,5,95,2.5,21.0,21.0,23.0,24.0,24.0,25.0,...,5,5,5,5,4,5,4,14.5,17.0,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,great pyrenees,1,30,60.5,69.0,75.0,67.0,67.0,66.0,66.0,...,3,5,3,3,3,3,3,29.5,107.5,giant
101,scottish terrier,1,30,60.5,55.0,59.0,58.0,58.0,58.0,57.0,...,4,5,4,3,3,4,4,11.0,20.0,small
102,bull terrier,1,30,60.5,52.0,53.0,55.0,57.0,60.0,62.0,...,4,3,3,3,4,3,4,21.5,60.0,large
103,chihuahua,1,30,60.5,22.0,24.0,28.0,30.0,32.0,33.0,...,4,4,4,3,4,5,3,7.5,3.5,teacup


In [37]:
# df_dogs.to_csv('dogs.csv', index=False)

## Download Breed Images

In [44]:
for breed, url in df_dogs[['breed', 'image']].to_numpy():
    img_data = requests.get(url).content
    with open(f'./images/{"_".join(breed.split())}.jpg', 'wb') as handler:
        handler.write(img_data)